In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)

data.head(1)

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama


In [2]:
# Q1: An aggregate of the movie ratings for each particular genre. 
mean_ratings = data.pivot_table('rating', index='genres', aggfunc='mean')

mean_ratings[:5]

genres
Action                                           3.354886
Action|Adventure                                 3.676814
Action|Adventure|Animation                       4.147826
Action|Adventure|Animation|Children's|Fantasy    2.703704
Action|Adventure|Animation|Horror|Sci-Fi         3.546926
Name: rating, dtype: float64

In [3]:
import numpy as np
# Q2: The top 5 highest ranked genre by women. 
movie_stats = data[data['gender'] == 'F'].groupby('genres').agg({'rating': np.mean})
movie_stats.sort_values(['rating'], ascending=False).head()

,rating
genres,
Animation|Comedy|Thriller,4.550802
Animation,4.533333
Film-Noir|Romance|Thriller,4.448718
Sci-Fi|War,4.376623
Adventure|Children's|Drama|Musical,4.355030


In [5]:
# Q3: The top 5 highest ranked genre by men. 
movie_stats = data[data['gender'] == 'M'].groupby('genres').agg({'rating': np.mean})
movie_stats.sort_values(['rating'], ascending=False).head()

,rating
genres,
Sci-Fi|War,4.464789
Animation|Comedy|Thriller,4.445110
Film-Noir|Mystery,4.381536
Adventure|War,4.376633
Animation,4.353107


In [6]:
# Q4: A breakdown of a movie’s ratings by age, use any movie of your choice.
# Here I use movie_id = 661 as the target movie
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
data['age_group'] = pd.cut(data.age, range(0, 81, 10), right=False, labels=labels)
data[data['movie_id'] == 661].groupby('age_group')['rating'].agg({'average rating': np.mean})

,average rating
age_group,
0-9,3.085714
10-19,3.693694
20-29,3.463768
30-39,3.438776
40-49,3.368421
50-59,3.305556
60-69,NaN
70-79,NaN


In [10]:
# Q5: A function that given a user_id and movie_id , returns a list of user_ids for other users 
# that rated the movie identified by the provided movie_id with the same score. 
def get_users_with_the_same_score(uid, mid):
    movie_rating = data[(data['user_id'] == uid) & (data['movie_id'] == mid)].get_value(0,'rating')
    movie_info = data[(data['movie_id'] == mid) & (data['rating'] == movie_rating) & (data['user_id'] != uid)]
    return movie_info['user_id'].tolist()

print get_users_with_the_same_score(1, 1193)

[2, 17, 19, 24, 33, 39, 53, 54, 58, 81, 88, 89, 95, 102, 107, 120, 123, 131, 150, 155, 166, 169, 175, 180, 190, 195, 208, 235, 236, 259, 261, 265, 266, 270, 281, 287, 292, 293, 297, 299, 300, 301, 306, 307, 315, 319, 327, 332, 334, 339, 352, 355, 357, 361, 409, 424, 425, 437, 438, 439, 440, 442, 445, 451, 453, 462, 477, 480, 482, 489, 491, 502, 504, 505, 507, 515, 516, 520, 524, 526, 529, 537, 549, 550, 551, 558, 560, 582, 588, 599, 611, 622, 626, 637, 650, 661, 671, 673, 677, 679, 681, 682, 692, 701, 710, 711, 743, 744, 746, 752, 760, 764, 770, 771, 774, 784, 785, 791, 833, 839, 840, 841, 843, 846, 870, 900, 919, 928, 935, 946, 957, 965, 980, 983, 994, 1010, 1015, 1017, 1018, 1019, 1027, 1028, 1035, 1036, 1038, 1040, 1048, 1058, 1062, 1068, 1077, 1084, 1088, 1098, 1099, 1101, 1105, 1108, 1113, 1117, 1121, 1124, 1133, 1134, 1165, 1170, 1172, 1178, 1180, 1182, 1197, 1202, 1203, 1211, 1216, 1218, 1224, 1225, 1228, 1239, 1243, 1260, 1264, 1267, 1273, 1285, 1289, 1303, 1305, 1324, 1331, 13

In [14]:
# Q6: Some other statistic, figure, data aggregate, or plot that you created using this dataset, 
# along with a short description of what interesting observations you derived from it.

# Given an occupation id, what's the top 5 most highly rate movie genres for that occupation?
data[data['occupation'] == 11].groupby('genres').agg({'rating': np.mean}).sort_values('rating', ascending=False).head()

,rating
genres,
Drama|Fantasy,5.000000
Children's|Sci-Fi,5.000000
Action|Adventure|Animation,4.800000
Animation,4.571429
Animation|Comedy|Thriller,4.571429


It seems like across different occupations, almost all viewers love Sci-Fi and Animation movies. It's actually pretty reasonable.

In [19]:
# Q6: Some other statistic, figure, data aggregate, or plot that you created using this dataset, 
# along with a short description of what interesting observations you derived from it.

# Show me the top 5 MOST "generous" movie grader by occupation
data.groupby('occupation').agg({'rating': np.mean}).sort_values('rating', ascending=False)[:5]

# Show me the top 5 LEAST "generous" movie grader by occupation
data.groupby('occupation').agg({'rating': np.mean}).sort_values('rating', ascending=True)[:5]

,rating
occupation,
19,3.414050
8,3.466741
20,3.497392
18,3.530117
10,3.532675


Due to the fact that I cannot retrieve the occupation name from id, I can only conclude that the most "generous" grader by occupation is occupation number 13. The average rating is 3.781. 

The least generous grader goes to occupation number 19. The average rating is 3.414. 

These are not dramatic differences, but interesting enough results that may cause us attention. It'd be even better if we can have the occupation id - name map.